# Preprocesado de datos

El preprocesado de datos es una fase indispensable para el correndo aprendizaje por partes de los algoritmos de Deeplearning. Se ha demostrado empíricamente que una correcta preparación y normalización de los datos permiten hallar soluciones más cercanas a la optima que con datos no procesados.

Es importante tener en cuenta que no existe una metodología de preprocesado única, y que es necesario adaptarse al tipo de dato que estamos tratando. Para este proyecto, además, existe una dificultad adicional, y es la existencia de diferentes procedencias para los datos, pues en total se dispone de 5 datasets distintos, cada uno recopilado con diferentes metodologías e instrumentación. Por tanto, será clave adaptarse a cada uno de los destinos, y realizar la partición final de forma estratificada para evitar sesgos que perturben el resutado.

Más adelante, profundizaremos en este aspecto, pero en primer lugar, debemos leer cada uno de los conjuntos de datos disponibles, y examinar de cuántos elementos disponemos en cada uno, para establecer la proporción de entrenamiento-test oportuna.

In [2]:
# Librerías utilizadas por el script
import os
import cv2
import zipfile
import csv
import pathlib

import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns

from copy import deepcopy

## Dataset disponibles

Haciendo uso de los recursos disponibles públicamente, se han tomado los siguientes datasets para realizar el experimento:
- ISIC: es el mayor conjuntos de datos cutáneos disponible en abierto, y contiene imágenes de todo tipo de pieles y procedencias, pero con especial énfasis en las personas de origen europeo y americano.
- ASAN: Conjunto de datos provenientes del hospital con este mismo nombre, con lesiones en personas de origen asiático.
- PAD UFES 20: conjunto de datos de lesiones variadas de pacientes latinoamericanos.
- PH2: banco de datos de pacientes brasileños con lesiones potencialmente cancerosas.
- Severance: base de datos con lesiones cutáneas en población asiática, con contenido tanto benigno como cancerígeno.

Para unificar la notación de los datos, se creará un código para la notación de cada una de las clases que permitan un procesamiento común de todos los datos sin depender del origen de este. Para ello, se creará un nuevo archivo .csv donde se anotará el path de la imagen, su clase asociada, y el tipo general de la misma (beningna o maligna). La metainformación asociada, de momento, quedará relegado a un segundo plano hasta el estudio estadístico de los datos.

In [3]:
# Directorios de cada dataset

ISIC_PATH = "datasets/ISIC"
ASAN_PATH = "datasets/Asan"
PAD_UFES_PATH = "datasets/PAD_UFES_20"
PH2_PATH = "datasets/PH2"
SEVERANCE = "datasets/Severance"

In [4]:
# Funciones comunes para la lectura y muestra de datos

'''
This function receives a string with the filename of the image to read,
and a flag indicating if we want to read it in color/RGB (flagColor=1) or gray level (flagColor=0)

Example of use:
im1=readIm(get_image('apple.jpg'),0)
'''


def readIm(filename, flagColor=1):
    # cv2 reads BGR format
    im = cv2.imread(filename)
    # change to  RGB and return the image
    if (flagColor):
        return cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    # change from BGR to grayscale instead if flag is 0
    return cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)


'''
This function receives an array of arbitrary real numbers (that could include even negative values),
and returns an 'image' in the range [0,1].
flag_GLOBAL allows the user to normalize the whole image (including all channels) or to normalize
each channel/band independently.
'''


def rangeDisplay01(im, flag_GLOBAL=True):
    im = im.astype(float)
    if flag_GLOBAL:
        im = (im - im.min()) / (im.max() - im.min())
    else:
        # bands normalization
        for band in range(im.shape[2]):
            im[:, :, band] = (im[:, :, band] - im[:, :, band].min()) / (im[:, :, band].max() - im[:, :, band].min())
            # Note: remember that, for plt.imshow with RGB data, the valid range is [0..1] for floats and [0..255] for integers.
    return im


"""
Función para mostrar imágenes en pantalla en color y blanco negro. Permite realizar
aumento sobre las mismas para apreciar un mayor detalle.

Entrada:
    im: imagen leída en formato ndarray
    title: nombre que recibe el marco en pantalla
    factor: factor de aumento de la image, "zoom"
"""


def displayIm(im, title='Result', factor=2):
    # First normalize range
    max = np.max(im)
    min = np.min(im)
    if min < 0 or max > 255:
        im = rangeDisplay01(im, flag_GLOBAL=True)
    if len(im.shape) == 3:
        # im es tribanda
        plt.imshow(im, cmap='jet')
    else:
        # im es monobanda
        plt.imshow(im, cmap='gray')
    figure_size = plt.gcf().get_size_inches()
    plt.gcf().set_size_inches(factor * figure_size)
    plt.title(title)
    plt.xticks([]), plt.yticks([])  # eliminamos numeración
    plt.show()

In [5]:
# En estas variables, se acumularán las clases de cada dataset para matener notación común

global_y = []

### ISIC Skin Dataset

Se trata del dataset de mayor tamaño del conjunto. Contiene 31 clases identificadas, tanto de lesiones benignas y malignas de la piel. En total, se dispone de 53738, las cuales ya han sido filtradas para asegurarse de que no exista redundancia por las herramientas online de la galería ISIC: https://gallery.isic-archive.com/

In [6]:
def extractISIC(path: str):
    # Obtener una lista de todos los archivos ZIP en la carpeta especificada
    archivos_zip = [f for f in os.listdir(path) if f.lower().endswith('.zip')]

    # Iterar sobre cada archivo ZIP
    for archivo_zip in archivos_zip:
        ruta_archivo_zip = os.path.join(path, archivo_zip)
        carpeta_salida = os.path.splitext(ruta_archivo_zip)[0]  # Eliminar la extensión .zip

        # Comprobar si la carpeta de salida ya existe
        if not os.path.exists(carpeta_salida):
            os.makedirs(carpeta_salida)  # Crear la carpeta de salida

            # Extraer el contenido del archivo ZIP en la carpeta de salida
            with zipfile.ZipFile(ruta_archivo_zip, 'r') as zip_ref:
                zip_ref.extractall(carpeta_salida)
            print(f"Extraído {archivo_zip} en {carpeta_salida}")
        else:
            print(f"Omitido {archivo_zip}. {carpeta_salida} ya existe.")


def listar_clases(path):
    # Obtener una lista de todas las carpetas en el path
    return [nombre for nombre in os.listdir(path) if os.path.isdir(os.path.join(path, nombre))]


def definir_etiquetas_ISIC():
    clases = listar_clases(ISIC_PATH)
    print(clases)
    return [clase.replace(" ", "_").lower() for clase in clases]


def crear_csv(path, clases, nombre_dataset):
    # Inicializa una lista vacía para almacenar la información de los archivos
    info_archivos = []
    i = 0

    # Itera sobre cada carpeta en la carpeta raíz
    for nombre_carpeta in os.listdir(path):
        ruta_carpeta = os.path.join(path, nombre_carpeta)

        if os.path.isdir(ruta_carpeta):

            # Itera sobre cada archivo en la carpeta
            for nombre_archivo in os.listdir(ruta_carpeta):
                ruta_archivo = os.path.join(ruta_carpeta, nombre_archivo)
                if os.path.isfile(
                        ruta_archivo) and nombre_archivo != "metadata.csv" and nombre_archivo != "attribution.txt":
                    # Agrega la información del archivo a la lista
                    info_archivos.append((nombre_archivo, ruta_archivo, clases[i], nombre_dataset))
            i += 1

    # Define la ruta del archivo CSV
    ruta_archivo_csv = "preprocessedData.csv"

    # Escribe la información de los archivos en el archivo CSV
    with open(ruta_archivo_csv, "w", newline="") as archivo_csv:
        escritor_csv = csv.writer(archivo_csv)
        escritor_csv.writerow(["image", "dir", "class", "dataset"])  # Escribe la cabecera
        escritor_csv.writerows(info_archivos)  # Escribe la información de los archivos

In [7]:

# Accedemos al directorio de ISIC
print(os.getcwd())

# Extraemos cada zip, en caso de que no exista
#extractISIC(ISIC_PATH)

# Tomamos los nombres de cada imagen, y le asociamos su etiqueta manualmente
print(os.getcwd())

isic_y = definir_etiquetas_ISIC()
global_y = deepcopy(isic_y)

# Creamos CSV
crear_csv(ISIC_PATH, isic_y, "ISIC")
print(os.getcwd())

C:\Users\Cris1\Documents\TFG
C:\Users\Cris1\Documents\TFG
[]
C:\Users\Cris1\Documents\TFG


### ASAN Dataset

Este dataset es el segundo de mayor tamaño recogido. En total, dispone de 17,125 imágenes de 12 clases distintas. Muchas de estas clases tratan la misma enfermedad, pero distinguen si se ha realizado biopsia o no para verificarlo (aunque todos los resultados han sido confirmados posteriormente tras estudiar su evolución).

La dificultad de este conjunto de datos se debe al formato de almacenaje escogido: todas las imágenes fueron guardadas en estructura de rejilla, provocando la existencia de cientos de imágenes en un mismo espacio separado por bordes blancos. Por tanto, habrá que realizar una etapa de preprocesado más profundo que ISIC para dividir correctamente la imagen y remover los bordes blancos para evitar sesgos en los resultados del modelo final.

El código se divide en dos fases: una primera fase de separación, y la segunda de barajado, donde uniremos el conjunto de entrenamiento y test dividido anteriormente, ya que la proporción elegida para test fue de un apenas 10%, y no se conoce el grado de aleatoriedad del criterio de seperación elegido.

In [15]:
grosor_borde = 8  # Constante del borde a eliminar (px)
carpeta_dest = "thumbnails"


# Recortado
def recortarImagenesASAN(path):
    os.chdir(path)

    if not os.path.exists(carpeta_dest):
        os.makedirs(carpeta_dest)

    files = [f for f in pathlib.Path().iterdir() if f.is_file()][1:]

    names = []
    diss_class = []
    for f in files:
        name = str(f)[20:-4]
        if ".png" in str(f):
            image = cv2.imread(str(f), cv2.IMREAD_UNCHANGED)

            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
            gradient = cv2.morphologyEx(gray, cv2.MORPH_GRADIENT, kernel)

            contours, _ = cv2.findContours(gradient, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            i = 0
            for cnt in contours:
                (x, y, w, h) = cv2.boundingRect(cnt)
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 0))
                box_image = image[y: y + h, x: x + w]

                # Recorta la imagen para eliminar el borde
                img_sin_borde = box_image[grosor_borde:-grosor_borde, grosor_borde:-grosor_borde]
                cv2.imwrite(f"thumbnails/{name}_{i}.png", img_sin_borde)
                i += 1

                names.append(f"{name}_{i}.png")
                i += 1
                diss_class.append(name)

    os.chdir("../../../")

In [17]:
# Ejecutamos la separación del conjunto de train
print(os.getcwd())

recortarImagenesASAN(ASAN_PATH + "/train_dataset")
recortarImagenesASAN(ASAN_PATH + "/test_dataset")
recortarImagenesASAN(ASAN_PATH + "/Hallym_dataset")

C:\Users\Cris1\Documents\TFG
im_test.ipynb
L#12dx#test-asan10#biopsy#ak.png
L#12dx#test-asan10#biopsy#ak.png
L#12dx#test-asan10#biopsy#bcc.png
L#12dx#test-asan10#biopsy#bcc.png
L#12dx#test-asan10#biopsy#dermatofibroma.png
L#12dx#test-asan10#biopsy#dermatofibroma.png
L#12dx#test-asan10#biopsy#hemangioma.png
L#12dx#test-asan10#biopsy#hemangioma.png
L#12dx#test-asan10#biopsy#intraepithelial carcinoma.png
L#12dx#test-asan10#biopsy#intraepithelial carcinoma.png
L#12dx#test-asan10#biopsy#lentigo.png
L#12dx#test-asan10#biopsy#lentigo.png
L#12dx#test-asan10#biopsy#melanoma.png
L#12dx#test-asan10#biopsy#melanoma.png
L#12dx#test-asan10#biopsy#nevus.png
L#12dx#test-asan10#biopsy#nevus.png
L#12dx#test-asan10#biopsy#pyogenic granuloma.png
L#12dx#test-asan10#biopsy#pyogenic granuloma.png
L#12dx#test-asan10#biopsy#scc.png
L#12dx#test-asan10#biopsy#scc.png
L#12dx#test-asan10#biopsy#sebk.png
L#12dx#test-asan10#biopsy#sebk.png
L#12dx#test-asan10#biopsy#wart.png
L#12dx#test-asan10#biopsy#wart.png
L#12dx#